# Chapter4 - Exercise 2: Play Goft
# Cho dữ liệu play golf trong tập tin playgoft_data.xlsx.

## Yêu cầu: Hãy đọc dữ liệu từ tập tin này, áp dụng Naive Bayes để thực hiện việc xác định có đi chơi golf hay không dựa trên các thông tin như: 
+ 'Outlook'
+ 'Temperature'
+ 'Humidity'
+ 'Wind'
+ 'Play Golf'


## Yêu cầu:
1. Hãy chuẩn hóa dữ liệu cho phù hợp
2. Áp dụng Naive Bayes. Tìm kết quả
3. Kiểm tra độ chính xác
4. Xuất/ghi model
5. Đọc model
6. Cho dữ liệu Test: X_test = [["Overcast", "Cool", "High", "Strong"], ["Sunny", "Cool", "High", "Weak"]] => Y_test

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn

np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows',10)

## <span style='color:yellow'> 1. Hãy chuẩn hóa dữ liệu cho phù hợp

In [11]:
golf = pd.read_excel('Data/playgolf_data.xlsx', index_col=0)
golf

,Outlook,Temperature,Humidity,Wind,Play Golf
Day,,,,,
1,Sunny,Hot,High,Weak,No
2,Sunny,Hot,High,Strong,No
3,Overcast,Hot,High,Weak,Yes
4,Rain,Mild,High,Weak,Yes
5,Rain,Cool,Normal,Weak,Yes
...,...,...,...,...,...
10,Rain,Mild,Normal,Weak,Yes
11,Sunny,Mild,Normal,Strong,Yes
12,Overcast,Mild,High,Strong,Yes


In [13]:
golf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14 entries, 1 to 14
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Outlook      14 non-null     object
 1   Temperature  14 non-null     object
 2   Humidity     14 non-null     object
 3   Wind         14 non-null     object
 4   Play Golf    14 non-null     object
dtypes: object(5)
memory usage: 672.0+ bytes


In [15]:
golf.isnull().sum()

Outlook        0
Temperature    0
Humidity       0
Wind           0
Play Golf      0
dtype: int64

In [17]:
golf['Play Golf'].value_counts()

Play Golf
Yes    9
No     5
Name: count, dtype: int64

## <span style='color:yellow'> 2. Áp dụng Naive Bayes. Tìm kết quả
## <span style='color:yellow'> 3. Kiểm tra độ chính xác

In [25]:
x = golf.drop('Play Golf', axis=1)
y = golf['Play Golf']

In [36]:
from sklearn.naive_bayes import BernoulliNB
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

Onehot_transformer = ColumnTransformer(
    transformers=[('Onehot_Encoder', OneHotEncoder(handle_unknown='ignore'), x.columns)],
    remainder='passthrough'
)

nbBer_pipe = Pipeline([
    ('Onehot_Encoder', Onehot_transformer),
    ('Bernoulli_Naive_Bayes', BernoulliNB())
])

nbBer_pipe.fit(x,y)

print(f'BernoulliNB accuracy score on train set: {nbBer_pipe.score(x, y)}')

BernoulliNB accuracy score on train set: 0.9285714285714286


## <span style='color:yellow'> 4. Xuất/ghi model
## <span style='color:yellow'> 5. Đọc model

In [54]:
import pickle
pkl_filename = 'model_ex2_BernoulliNB.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(nbBer_pipe, file)
file.close()

In [58]:
file = open(pkl_filename, 'rb')
nbBer_model = pickle.load(file)
file.close()

## <span style='color:yellow'> 6. Cho dữ liệu Test: X_test = [["Overcast", "Cool", "High", "Strong"], ["Sunny", "Cool", "High", "Weak"]] => Y_test

In [67]:
X_test = pd.DataFrame([["Overcast", "Cool", "High", "Strong"], ["Sunny", "Cool", "High", "Weak"]],
                     columns = x.columns)

Y_test = nbBer_model.predict(X_test)

X_test['Play_golf_Predict'] = Y_test

X_test

,Outlook,Temperature,Humidity,Wind,Play_golf_Predict
0,Overcast,Cool,High,Strong,Yes
1,Sunny,Cool,High,Weak,No
